# Notebook to create the two-dimensional labels by overlaying the After RGB GeoTIFF and the curated fire polygons 
## Import the necessary libraries and packages

In [1]:
import io
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import box 
from shapely.geometry import shape
import rasterio.features
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
import matplotlib.pyplot as plt



## Set Google Drive access

In [2]:
# OAuth 2.0 credentials (client_secret.json) should be in the same directory as this script
CLIENT_SECRET_FILE = 'client_secrets.json'
API_NAME = 'drive'
API_VERSION = 'v3'

# Create OAuth flow
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, ['https://www.googleapis.com/auth/drive'])

# Authenticate and authorize the user
credentials = flow.run_local_server(port=0)

# Create a Drive API client
service = build(API_NAME, API_VERSION, credentials=credentials)

# List files in your Google Drive
results = service.files().list().execute()
files = results.get('files', [])
# Define the folder IDs for "after" RGB images in Google Drive
after_folder_id = '1zd9Wx7GXqw-w9F4hDZyViF9Rq04K5veI'

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1003083568572-78avuh89fu558ciatmapqhgpka9p4c3l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56611%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=yQXasA0axwj7GEJVNRpcoO5gNIr05Y&access_type=offline


## List files in folder

In [3]:
# Function to list all files in a folder with their IDs and names
def list_files_in_folder(service, folder_id):
    results = []
    page_token = None

    while True:
        response = service.files().list(
            q=f"'{folder_id}' in parents",
            pageSize=1000,  # Increase if you have more than 1000 files
            pageToken=page_token,
            fields="nextPageToken, files(id, name)"
        ).execute()

        files = response.get('files', [])
        results.extend(files)
        page_token = response.get('nextPageToken')
        if not page_token:
            break

    return results
# List all files in the "after" folder
after_files = list_files_in_folder(service, after_folder_id)
#Create a python list with the names 
after_filenames = [file['name'] for file in after_files]

## Code to create, overlay and upload the masks

In [4]:
# Define the paths to your Shapefile
shapefile_path = 'PolygonFiles/FirePolygons.shp'

# Read the Shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# Reproject the shapefile to EPSG:4326
gdf = gdf.to_crs(epsg=4326)

# Create a dictionary to store binary masks for each image
image_masks = {}

# Google Drive folder ID for storing mask results
mask_folder_id = '1ky2Jj7s7jnyeVWVwPHEyrYE-mOlsu1oO'

# Function to upload a file to Google Drive
def upload_tiff_to_drive(service, file_content, file_name, folder_id):
    media = MediaIoBaseUpload(io.BytesIO(file_content), mimetype='image/tiff')
    request = service.files().create(
        media_body=media,
        body={'name': file_name, 'parents': [folder_id]}
    )
    request.execute()

# Iterate through the rows in the attribute table
for i, (index, row) in enumerate(gdf.iterrows()):
    # Skip rows before index x in case code fails at a certain point
    if i < 366:
        continue
    object_id = row['ORIGINALOI']
    image_name = f'RGB_AfterFire{object_id}.tif'

    # Check if the image exists in Google Drive
    file_id = None
    for file in after_files:
        if file['name'] == image_name:
            file_id = file['id']
            break

    if file_id:
        # Download the image from Google Drive
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            _, done = downloader.next_chunk()

        # Read the downloaded image using rasterio
        fh.seek(0)
        with rasterio.open(fh) as src:
            image_width = src.width
            image_height = src.height
            image_transform = src.transform
            image_bounds = box(src.bounds[0], src.bounds[1], src.bounds[2], src.bounds[3])

        geom = shape(row['geometry'])  # Extract the geometry column
        clipped_geom = geom.intersection(image_bounds.envelope)

        if not clipped_geom.is_empty:
            # Create a binary mask by rasterizing the clipped geometry
            clipped_mask = rasterio.features.geometry_mask([clipped_geom], out_shape=(image_height, image_width), transform=image_transform, invert=True)

            # Combine the clipped mask with the mask for the specific image
            if image_name not in image_masks:
                image_masks[image_name] = np.zeros((image_height, image_width), dtype=np.uint8)
            
            image_masks[image_name] = np.maximum(image_masks[image_name], clipped_mask)

            # Create the temporary directory if it doesn't exist
            temp_dir = 'temp_masks'
            if not os.path.exists(temp_dir):
                os.makedirs(temp_dir)

            # Save the image with the mask overlay
            temp_output_path = f'{temp_dir}/{image_name[:-4]}_with_mask.tif'  # Temporary output image path with mask overlay
            with rasterio.open(temp_output_path, 'w', driver='GTiff', width=image_width, height=image_height, count=1, dtype=np.uint8, crs=src.crs, transform=image_transform) as dst:
                dst.write(clipped_mask, 1)

            # Upload the result to Google Drive
            upload_tiff_to_drive(service, open(temp_output_path, 'rb').read(), f'{image_name[:-4]}_with_mask.tif', mask_folder_id)
            
            # Remove the temporary file
            os.remove(temp_output_path)

